# Suche nach Plagiaten mit Python's `SequenceMatcher`

`test/test1` und `test/test2` sind mögliche Plagiate, während `test/test3` gänzlich anders ist.
Der Parameter `threshold` muss so eingestellt werden, damit es richtig erkannt wird!

In [ ]:
from difflib import SequenceMatcher, HtmlDiff

In [ ]:
text1 = """\
First-order methods can significantly benefit from parallel computing. These computing systems are
typified by uniform processing nodes that are in close proximity and have reliable communications.
Indeed, the expression embarrassingly parallel refers to an ideal scenario for parallelization where
we split the job into independent calculations that can be simultaneously performed in a predictable
fashion.
"""

In [ ]:
text2 = """\
First-order methods can significantly benefit from parallel computing. These computing systems are
typified by uniform processing nodes that are in close proximity and have reliable communications.
Indeed, the expression parallel as embarrassingly refers to an ideal scenario for parallelization where
we split the job into independent calculations thit can be simultaneously performed in a predictable
fashion.
"""

In [ ]:
sm = SequenceMatcher(a=text1, b=text2)
print sm.real_quick_ratio()
print sm.quick_ratio()
print sm.ratio()

## Iteriere über alle IPython Dateien in einem gegeneben Verzeichnis (mit Unterverzeichnissen)

In [ ]:
import os
import itertools as it
from IPython.nbformat import current
from difflib import SequenceMatcher
from IPython.display import display, HTML

In [ ]:
def extract(fn):
    with open(fn, "r") as f:
        nb = current.read(f, "ipynb")
        code = []
        text = []
        for cell in nb["worksheets"][0]["cells"]:
            if cell["cell_type"] == "code":
                code.append(cell["input"] + "\n")
            elif cell["cell_type"] == "markdown":
                text.append(cell["source"])
        return "\n".join(code), "\n".join(text)

In [ ]:
print extract("./test/test1.ipynb")

In [ ]:
def distance(x1, x2, threshold=0.8):
    sm = SequenceMatcher(a=x1, b=x2)
    # quick and real_quick are *upper* bounds, hence threshold check from below
    rqr = sm.real_quick_ratio()
    if rqr > threshold:
        qr = sm.quick_ratio()
        if qr > threshold:
            return sm.ratio()
        return qr
    return rqr

In [ ]:
distance(text1, text2)

In [ ]:
HTML("""\
<style>
table.plagiate tr td {
    font-size: 70%;
    color: #444;
}
table.plagiate tr.hit {
    font-weight: bold;
}
table.plagiate tr.hit td { 
    background: red;
    color: white;
    font-size: 100%;
    padding: 4px;
}
.rendered_html table.plagiate tr td:nth-child(3) {
   text-align: right;
}
.rendered_html table.plagiate td {
   padding: 1px 4px;
   border-color: #888;
}
</style>
""")

In [ ]:
from matplotlib.colors import rgb2hex
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
from numpy import log1p, exp, linspace
import numpy as np
import sys

In [ ]:
def color_mapping(x, k=3):
    """
    logistic curve to map the distance metric [0, 1] to [0, 1] in the color space
    """
    return x**k / exp(1-x**k)

In [ ]:
xx = linspace(0, 1, 100)
plt.plot(xx, color_mapping(xx))

In [ ]:
colors = plt.cm.Reds
text_distances = []
code_distances = []

from IPython.display import clear_output

def plagiate(roots, threshold = .75, show_all=False):
    codes = {}
    texts = {}
    output = ""
    for root in roots:
        root = os.path.abspath(root)
        for path, _, filenames in os.walk(root):
            if ".ipynb_checkpoints" in path:
                continue
            for filename in filenames:
                if not filename.endswith(".ipynb"):
                    continue
                fn = os.path.normpath(os.path.join(root, path, filename))
                try:
                    codes[fn[len(root)+1:]] = extract(fn)
                except:
                    pass
                
                if np.random.random() > .7:
                    clear_output(wait=True)
                    print "processing %s ..." % fn[len(root) + 1:]
                    sys.stdout.flush()
    
    clear_output(wait=True)
    similar = nx.Graph()
    
    pairs = it.combinations(sorted(codes.iteritems()), 2)
    nbpairs = (len(codes) * (len(codes) - 1)) / 2
    i = 0
    for (k1, (c1, t1)), (k2, (c2, t2)) in pairs:
        name1, name2 = ' '.join(k1.split("_")[:2]), ' '.join(k2.split("_")[:2])
        #print levenshtein.ratio(c1, c2)
        if i % 10 == 0:
            clear_output(wait=True)
            print "pairing %4.1f%%  %30s <-> %-30s" % (100. * i / nbpairs, name1, name2)
            sys.stdout.flush()
        i += 1
        
        d_code = distance(c1, c2, threshold)
        d_text = distance(t1, t2, threshold)
        
        code_distances.append(d_code)
        text_distances.append(d_text)
        
        d = min(d_code, d_text)
        color = rgb2hex(colors(color_mapping(d)))
        if not show_all and d < threshold:
            continue
        
        similar.add_edge(name1, name2, weight=d)
               
        cls = "class='hit'" if d >= threshold else ''
        output += "<tr {} style='background:{}'>\
            <td>{}</td><td>{}</td><td>{:4f}</td><td>{:4f}</td></tr>".format(cls, color, k1, k2, d_code, d_text)
        
        
    clear_output()
    display(HTML("""<table class='plagiate'>
                 <tr><th>file1</th><th>file2</th><th>d(code)</th><th>d(text)</th></tr>""" 
                 + output + "</table>"))
    
    fig = plt.figure()
    f1 = fig.add_subplot(2,1,1)
    f2 = fig.add_subplot(2,1,2)
    _ = f1.hist(code_distances)
    _ = f2.hist(text_distances)
               
    return similar

In [ ]:
similar = plagiate(roots=["../../ss2015/abgaben/Abgabe10Gruppe1/",
                          "../../ss2015/abgaben/Abgabe11Gruppe2/"],
                   threshold=.85,
                   show_all=False)

In [ ]:
plt.rcParams["figure.figsize"] = (10,8)
nx.draw(similar, with_labels=True, iterations=200,
               font_color="blue", node_color="white", edge_color="grey", font_size=10)

In [ ]:
%config InlineBackend.figure_formats=['png']
plt.rcParams["figure.figsize"] = (12,10)
nx.draw_graphviz(similar, with_labels=True, iterations=200,prog="fdp",
               font_color="blue", node_color="white", edge_color="grey", font_size=10)

In [ ]:
def showdiff(fn1, fn2):
    htmldiff = HtmlDiff()
    c1, _ = extract(fn1)
    c2, _ = extract(fn2)
    return HTML(htmldiff.make_table(c1.splitlines(), c2.splitlines(), fn1, fn2, context=True))

In [ ]:
# style for the table
HTML("""\
<style>
.rendered_html > table.diff td,
.rendered_html > table.diff tr {
   border: none;
   font-size: 12px;
   padding: 1px 5px;
   text-align: right;
}
.rendered_html > table.diff td:nth-child(3),
.rendered_html > table.diff td:nth-child(6) {
   text-align: left;
   padding-right: 3em;
}
.rendered_html > table.diff tr:nth-child(even) td  {
  background: #eee;
}
table.diff tr > td {
  font-family: monospace;
}
</style>
""")

In [ ]:
showdiff("test/test1.ipynb", "test/test2.ipynb")

In [ ]:
showdiff("test/test1.ipynb", "test/test3.ipynb")